In [4]:
!git clone https://github.com/Chando0185/first-order-model


Cloning into 'first-order-model'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 54 (delta 7), reused 1 (delta 0), pack-reused 0
Unpacking objects: 100% (54/54), done.


In [6]:
!pip install PyYAML

In [7]:
cd first-order-model


/content/first-order-model


In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [25]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread('/content/gdrive/My Drive/first-order-motion-model/212.png')
driving_video = imageio.mimread('/content/gdrive/My Drive/first-order-motion-model/08.mp4')


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

In [26]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/first-order-motion-model/vox-cpk.pth.tar')

In [27]:
from demo import make_animation
from skimage import img_as_ubyte

predections = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predections])

HTML(display(source_image, driving_video, predections).to_html5_video())

100%|██████████| 150/150 [00:06<00:00, 23.70it/s]
